# Keras Tuner

El desarrollo de modelos de aprendizaje automático suele ser un proceso iterativo. Comienza con un diseño inicial y luego se reconfigura hasta que obtenga un modelo que pueda ser entrenado de manera eficiente en términos de tiempo y recursos de calcular. Como ya sabrá, estas configuraciones que ajusta se llaman híper parámetros. Estas son las variables que rigen el proceso de entrenamiento y la topología de un modelo ML. Estos permanecen constantes sobre el proceso de entrenamiento e impactan directamente el rendimiento de su programa ML.

El proceso de encontrar el conjunto óptimo de hiperparámetros se llama *ajuste de híper parámetros *o *hipertuning *, y es una parte esencial de un pipeline de aprendizaje automático. Sin ella, puede terminar con un modelo que tiene parámetros innecesarios y tarda demasiado en entrenar.

Los hiperparámetros son de dos tipos:
1. **Híper parámetros modelo** que influyen en la selección del modelo, como el número y el ancho de las capas ocultas

2. **Híper parámetros del Algoritmo** que influyen en la velocidad y la calidad del algoritmo de aprendizaje, como la tasa de aprendizaje para el descenso de gradiente estocástico (SGD) y el número de vecinos más cercanos para un clasificador de vecinos más cercanos (KNN).

Para modelos más complejos, el número de hiperparámetros puede aumentar dramáticamente y ajustarlos manualmente puede ser bastante desafiante.

En este Notebook, se presenta el ajuste de híper parámetros con [Keras Tuner](https://keras-team.github.io/keras-tuner/), un paquete Keras que automatiza este proceso. A modo de comparación, primero se entrena un modelo de referencia con hiperparámetros preseleccionados, luego rehacer el proceso con hiperparámetros ajustados. Algunos de los ejemplos y discusiones aquí se toman del [tutorial oficial proporcionado por Tensorflow](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb#scrollto=skwlozkpfgaj).

## Descargar y preparar el conjunto de datos

Primero cargamos el [conjunto de datos Mnist de moda](https://github.com/zalandoresearch/fashion-mnist). Utilizará esto para entrenar un modelo de aprendizaje automático que clasifica imágenes de ropa.

In [1]:
# Import keras
from tensorflow import keras

2023-03-09 18:13:41.534257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 18:13:41.776208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-09 18:13:41.776243: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-09 18:13:43.025869: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# Download the dataset and split into train and test sets
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


Para el preprocesamiento, se normalizan los valores de píxeles para que el entrenamiento converja más rápido.

In [3]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Rendimiento de línea de base

Como se mencionó, primero tendrá un rendimiento de línea de base utilizando parámetros seleccionados arbitrariamente para que pueda comparar los resultados más adelante. En aras del tiempo y los límites de recursos proporcionados, simplemente construirá una red neuronal densa superficial (DNN) como se muestra a continuación. Esto es para demostrar los conceptos sin involucrar enormes conjuntos de datos y largos tiempos de ajuste y entrenamiento. Como verá más adelante, incluso los modelos pequeños pueden tardar un tiempo en ajustarse. Puede extender los conceptos aquí cuando puede construir modelos más complejos en sus propios proyectos.

In [4]:
# Build the baseline model using the Sequential API
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(28, 28)))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1')) # You will tune this layer later
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(10, activation='softmax'))

# Print model summary
b_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2023-03-09 18:13:53.789967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-09 18:13:53.790008: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-09 18:13:53.790042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (91561e3127f9): /proc/driver/nvidia/version does not exist
2023-03-09 18:13:53.790396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Como se muestra, codificamos todos los hiperparámetros al declarar las capas. Estos incluyen el número de unidades ocultas, activación y abandono. Verá cómo puede ajustar automáticamente algunos de estos un poco más tarde.

Luego configuremos la pérdida, las métricas y el optimizador. La tasa de aprendizaje también es un hiperparámetro que puede ajustar automáticamente, pero por ahora, configuremos en `0.001`.

In [5]:
# Setup the training parameters
b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

Con todas las configuraciones establecidas, puede comenzar a entrenar el modelo. Hemos establecido el número de épocas en 10, pero no dude en aumentarlo si tiene más tiempo.

In [6]:
# Number of training epochs.
NUM_EPOCHS = 10

# Train the model
b_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

Epoch 1/10


2023-03-09 18:13:54.452162: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 150528000 exceeds 10% of free system memory.


1500/1500 [==============================] - 7s 4ms/step - loss: 0.5136 - accuracy: 0.8163 - val_loss: 0.4168 - val_accuracy: 0.8497
Epoch 2/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3926 - accuracy: 0.8568 - val_loss: 0.3822 - val_accuracy: 0.8619
Epoch 3/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3560 - accuracy: 0.8676 - val_loss: 0.3536 - val_accuracy: 0.8727
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3339 - accuracy: 0.8767 - val_loss: 0.3397 - val_accuracy: 0.8754
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3200 - accuracy: 0.8818 - val_loss: 0.3423 - val_accuracy: 0.8759
Epoch 6/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3057 - accuracy: 0.8868 - val_loss: 0.3232 - val_accuracy: 0.8837
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2949 - accuracy: 0.8909 - val_loss: 0.3184 - val_accuracy: 0.8840
Epo

Finalmente, desea ver cómo se desempeña este modelo de referencia en el conjunto de pruebas.

In [7]:
# Evaluate model on the test set
b_eval_dict = b_model.evaluate(img_test, label_test, return_dict=True)

313/313 [==============================] - 1s 2ms/step - loss: 0.3811 - accuracy: 0.8677


Definamos una función de ayuda para mostrar los resultados, por lo que es más fácil comparar más adelante.

In [8]:
# Define helper function
def print_results(model, model_name, eval_dict):
    '''
    Prints the values of the hyparameters to tune, and the results of model evaluation

    Args:
    model (Model) - Keras model to evaluate
    model_name (string) - arbitrary string to be used in identifying the model
    eval_dict (dict) -  results of model.evaluate
    '''
    print(f'\n{model_name}:')

    print(f'number of units in 1st Dense layer: {model.get_layer("dense_1").units}')
    print(f'learning rate for the optimizer: {model.optimizer.lr.numpy()}')

    for key,value in eval_dict.items():
        print(f'{key}: {value}')

# Print results for baseline model
print_results(b_model, 'BASELINE MODEL', b_eval_dict)


BASELINE MODEL:
number of units in 1st Dense layer: 512
learning rate for the optimizer: 0.0010000000474974513
loss: 0.38111504912376404
accuracy: 0.8676999807357788


Eso es todo para obtener los resultados para un solo conjunto de hiperparámetros. Como puede ver, este proceso puede ser tedioso si desea probar diferentes conjuntos de parámetros. Por ejemplo, ¿mejorará su modelo si usa `learning_rate = 0.00001` y` units = 128`?  El proceso será aún más difícil si decide también ajustar el dropout y probar otras funciones de activación también. Keras Tuner resuelve este problema al tener una API para buscar automáticamente el conjunto óptimo. Solo tendrá que configurarlo una vez que luego esperar los resultados.

## Keras Tuner

Para realizar hipertuning con Keras Tuner, deberá:

* Definir el modelo
* Seleccionar qué hiperparámetros sintonizar
* Definir su espacio de búsqueda
* Definir la estrategia de búsqueda

In [9]:
# Import required packages
import tensorflow as tf
import keras_tuner as kt

### Defina el modelo

El modelo que configuró para hipertuning se llama *hipermodelo *. Cuando construye este modelo, define el espacio de búsqueda de hiperparameter además de la arquitectura del modelo.

Puede definir un hipermodelo a través de dos enfoques:

* Mediante el uso de una función de constructor de modelos
* [Subclase de la clase `Hypermodel`](https://keras-team.github.io/keras-tuner/#you-can-use-a-hypermodel-subclass-instead-of-a-model-building--función) de la API de Keras Tuner


Acá se muestra el primer enfoque: utilizando una función de constructor de modelos para definir el modelo de clasificación de imágenes. Esta función devuelve un modelo compilado y utiliza hiperparámetros que define en línea para ajustar el modelo.

La función a continuación básicamente construye el mismo modelo que usó anteriormente. La diferencia es que hay dos hiperparámetros que están configurados para el ajuste:

* El número de unidades ocultas de la primera capa densa
* La tasa de aprendizaje del ADAM Optimizer

Verá que esto se hace con un objeto HyperParameters que configura el hiperparámetro que le gustaría ajustar. Para este ejercicio, lo harás:

* Se usa el método `int()` para definir el espacio de búsqueda para las unidades densas. Esto le permite establecer un valor mínimo y máximo, así como el tamaño de paso al incrementar entre estos valores.

* Se usa el método `Choice ()` para la tasa de aprendizaje. Esto le permite definir valores discretos para incluir en el espacio de búsqueda.

Puede ver todos los métodos disponibles y su uso de muestras en [Documentación oficial](https://keras-team.github.io/keras-tuner/documentation/hyperparameters/#hyperparameters).

In [10]:
def model_builder(hp):
    '''
    Builds the model and sets up the hyperparameters to tune.

    Args:
    hp - Keras tuner object

    Returns:
    model with hyperparameters to tune
    '''

    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu', name='dense_1'))

    # Add next layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

    return model

## Instanciar el Tuner y realizar hipertuning

Ahora que tiene el generador de modelos, puede definir cómo el Tuner puede encontrar el conjunto óptimo de híper parámetros, también llamado estrategia de búsqueda. Keras Tuner tiene [cuatro sintonizadores](https://keras-team.github.io/keras-tuner/documentation/tuners/) Disponible con estrategias incorporadas-`randomsearch`,` Hyperband`, `bayesianoptimization`, y` Sklearn`.

En este tutorial, usará el Tuner de hiperband. Hyperband es un algoritmo desarrollado específicamente para la optimización de los híper parametros. Utiliza la asignación de recursos adaptativos y la parada temprana para converger rápidamente en un modelo de alto rendimiento. Esto se realiza utilizando un soporte de estilo de campeonato deportivo en el que el algoritmo entrena una gran cantidad de modelos para algunas épocas y transporta solo la mitad de los modelos de alto rendimiento a la siguiente ronda. Puede leer sobre la intuición detrás del algoritmo en la Sección 3 de [este documento](https://arxiv.org/pdf/1603.06560.pdf).

Hyperband determina el número de modelos para entrenar en un soporte calculando 1 + log <Sub> `Factor` </sub> (` max_epochs`) y redondeándolo al entero más cercano. Verá estos parámetros (es decir, `factor` y` max_epochs` pasados al inicializador a continuación). Además, también deberá definir lo siguiente para instanciar el sintonizador de Hyperband:

* El hipermodelo (construido por la función de su constructor de modelos)
* El `Objective` para optimizar (por ejemplo, precisión de validación)
* Un `Directory` para guardar registros y puntos de control para cada prueba (configuración del modelo) se ejecuta durante la búsqueda de hiperparameter. Si vuelve a ejecutar la búsqueda de hiperparameter, el sintonizador Keras usa el estado existente de estos registros para reanudar la búsqueda. Para deshabilitar este comportamiento, pase un argumento adicional `sobrescribir = true` al instancias del Tuner.
* El `Project_Name` para diferenciar con otras ejecuciones. Esto se utilizará como un nombre de subdirectorio en el 'Directorio'.

Puede consultar la [documentación](https://keras.io/api/keras_tuner/tuners/hyperband/) para otros argumentos en los que puede transmitir.

In [12]:
# Instantiate the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='kt_dir',
                     project_name='kt_hyperband')

Veamos un resumen de los híper parámetros que ajustará:

In [13]:
# Display hypertuning settings
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


Puede pasar un callback dejar de entrenar antes cuando una métrica no está mejorando. A continuación, definimos un calback [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/earlystopping) para monitorear la pérdida de validación y dejar de entrenar si no está mejorando después de 5 epochs.

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Ahora ejecutará la búsqueda de híper parametros. Los argumentos para el método de búsqueda son los mismos que los utilizados para `tf.keras.model.fit` además del calback anterior. Esto tomará alrededor de 10 minutos en correr.

In [15]:
# Perform hypertuning
tuner.search(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 01m 00s]
val_accuracy: 0.8901666402816772

Best val_accuracy So Far: 0.8923333287239075
Total elapsed time: 00h 12m 47s
INFO:tensorflow:Oracle triggered exit


Puede obtener el modelo de mejor rendimiento con [get_best_hyperparameters()](https://keras-team.github.io/keras-tuner/documentation/tuners/#get_best_hyperparameters-method).

In [16]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



## Construir y entrenar el modelo

Ahora que tiene el mejor conjunto de hiperparámetros, puede reconstruir el hipermodelo con estos valores y volver a entrenarlo.

In [19]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 416)               326560    
                                                                 
 dropout_2 (Dropout)         (None, 416)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                4170      
                                                                 
Total params: 330,730
Trainable params: 330,730
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Train the hypertuned model
h_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5168 - accuracy: 0.8168 - val_loss: 0.4238 - val_accuracy: 0.8471
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3908 - accuracy: 0.8584 - val_loss: 0.3577 - val_accuracy: 0.8709
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3557 - accuracy: 0.8695 - val_loss: 0.3581 - val_accuracy: 0.8726
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3342 - accuracy: 0.8755 - val_loss: 0.3655 - val_accuracy: 0.8593
Epoch 5/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3182 - accuracy: 0.8813 - val_loss: 0.3396 - val_accuracy: 0.8760
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3057 - accuracy: 0.8864 - val_loss: 0.3361 - val_accuracy: 0.8798
Epoch 7/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2946 - accuracy: 0.8895 - val_loss: 0.3232 - val_accuracy:

Luego obtendrá su rendimiento contra el conjunto de pruebas.

In [21]:
# Evaluate the hypertuned model against the test set
h_eval_dict = h_model.evaluate(img_test, label_test, return_dict=True)

313/313 [==============================] - 1s 2ms/step - loss: 0.3557 - accuracy: 0.8723


Podemos comparar los resultados que obtuvimos con el modelo de referencia que utilizamos al comienzo. Los resultados pueden variar, pero generalmente obtendrá un modelo que tiene menos unidades en la capa densa, mientras que tiene pérdida y precisión comparables. Esto indica que redujo el tamaño del modelo y ahorró recursos de cómputo, al tiempo que tiene más o menos la misma precisión.

In [22]:
# Print results of the baseline and hypertuned model
print_results(b_model, 'BASELINE MODEL', b_eval_dict)
print_results(h_model, 'HYPERTUNED MODEL', h_eval_dict)


BASELINE MODEL:
number of units in 1st Dense layer: 512
learning rate for the optimizer: 0.0010000000474974513
loss: 0.38111504912376404
accuracy: 0.8676999807357788

HYPERTUNED MODEL:
number of units in 1st Dense layer: 416
learning rate for the optimizer: 0.0010000000474974513
loss: 0.35569459199905396
accuracy: 0.8723000288009644
